<a href="https://colab.research.google.com/github/fernandodeeke/epidemias/blob/main/seirs_diferencas_otimizado_fase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

# Função que simula o modelo SEIRS com beta sazonal
def simula_seirs(beta_base, beta_amplitude, sigma, gamma, omega, N, dt, days, initial_exposed, initial_infected):
    # Cálculo dos passos de tempo
    passos = int(days / dt) + 1
    tempo = np.linspace(0, days, passos)

    # Inicializa vetores para S, I, R (necessários para os retratos de fase)
    S = np.zeros(passos)
    I = np.zeros(passos)
    R = np.zeros(passos)
    E = initial_exposed
    S[0] = N - initial_exposed - initial_infected
    I[0] = initial_infected
    R[0] = 0.0

    for t in range(passos - 1):
        # Beta sazonal: varia com cosseno
        beta = beta_base * (1 + beta_amplitude * np.cos(2 * np.pi * tempo[t] / 365))

        # Atualização das variáveis
        new_exposed = beta * S[t] * I[t] / N
        new_infected = sigma * E
        new_recovered = gamma * I[t]
        new_susceptible = omega * R[t]

        S[t+1] = S[t] + (-new_exposed + new_susceptible) * dt
        E += (new_exposed - new_infected) * dt
        I[t+1] = I[t] + (new_infected - new_recovered) * dt
        R[t+1] = R[t] + (new_recovered - new_susceptible) * dt

    return S, I, R

# Função para criar a interface interativa
def interactive_seirs():
    # Define os botões deslizantes
    sliders = {
        "beta_base": FloatSlider(value=0.5, min=0.1, max=1.0, step=0.01, description='Beta Base'),
        "beta_amplitude": FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='Amplitude Beta'),
        "sigma": FloatSlider(value=1/7, min=0.01, max=1.0, step=0.01, description='Sigma'),
        "gamma": FloatSlider(value=1/5, min=0.01, max=1.0, step=0.01, description='Gamma'),
        "omega": FloatSlider(value=1/365, min=0.0, max=0.1, step=0.001, description='Omega'),
        "N": IntSlider(value=100000, min=1000, max=1000000, step=1000, description='População Total (N)'),
        "initial_exposed": IntSlider(value=10, min=0, max=1000, step=1, description='Expostos Iniciais'),
        "initial_infected": IntSlider(value=1, min=0, max=1000, step=1, description='Infectados Iniciais'),
        "days": IntSlider(value=1825, min=30, max=3650, step=1, description='Dias de Simulação'),
        "dt": FloatSlider(value=0.05, min=0.01, max=1.0, step=0.01, description='Passo de Tempo (dt)')
    }

    # Função para atualizar os gráficos
    def update_plot(**kwargs):
        # Executa a simulação
        S, I, R = simula_seirs(**kwargs)

        # Plota os retratos de fase
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))

        # Retrato de fase I(t) x S(t)
        axes[0].plot(S, I, lw=2, color='blue')
        axes[0].set_title("Retrato de Fase: I(t) x S(t)")
        axes[0].set_xlabel("Suscetíveis (S(t))")
        axes[0].set_ylabel("Infectados (I(t))")
        axes[0].grid()

        # Retrato de fase I(t) x R(t)
        axes[1].plot(R, I, lw=2, color='green')
        axes[1].set_title("Retrato de Fase: I(t) x R(t)")
        axes[1].set_xlabel("Recuperados (R(t))")
        axes[1].set_ylabel("Infectados (I(t))")
        axes[1].grid()

        plt.tight_layout()
        plt.show()

    # Cria a interface interativa
    interact(update_plot, **sliders)

# Executa a interface interativa
interactive_seirs()

interactive(children=(FloatSlider(value=0.5, description='Beta Base', max=1.0, min=0.1, step=0.01), FloatSlide…